# I. 데이터 프레임 변환

In [1]:
#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # warning 메시지 숨기기

import tensorflow as tf
print("Is GPU available:", tf.test.is_gpu_available())

import tensorflow as tf
print('tf.__version__:',tf.__version__)

2025-03-04 14:01:42.024265: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-04 14:01:42.074591: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-04 14:01:43.675174: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is GPU available: True
tf.__version__: 2.13.0


2025-03-04 14:01:46.026340: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-04 14:01:46.058759: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-04 14:01:46.060249: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
tf.config.set_visible_devices([], 'GPU')

2025-03-04 14:01:54.253600: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-04 14:01:54.255378: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-04 14:01:54.256812: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## 1. 교통소음 데이터 프레임 생성

In [4]:
import pymysql
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [5]:
import config 
from config import DB_CONFIG
import pymysql  
 
# MySQL 연결
conn = pymysql.connect(
     host=DB_CONFIG['host'],
     user=DB_CONFIG['user'],
     password=DB_CONFIG['password'],
     database=DB_CONFIG['database'],
     port=DB_CONFIG['port']
 )
 
car_siren = f"SELECT * FROM merge50_car_siren"
car_horn = f"SELECT * FROM merge50_car_horn"
car_driving = f'SELECT * FROM merge50_car_driving'
motorcycle_horn  = f'SELECT * FROM merge50_motorcycle_horn'
motorcycle_driving  = f'SELECT * FROM merge50_motorcycle_driving'
 
car_siren_df =  pd.read_sql(car_siren, conn)
car_horn_df = pd.read_sql(car_horn, conn) 
car_driving_df = pd.read_sql(car_driving, conn) 
motorcycle_horn_df= pd.read_sql(motorcycle_horn, conn) 
motorcycle_driving_df = pd.read_sql(motorcycle_driving, conn) 
 
conn.close()

/tmp/ipykernel_12980/30002709.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  car_siren_df =  pd.read_sql(car_siren, conn)
/tmp/ipykernel_12980/30002709.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  car_horn_df = pd.read_sql(car_horn, conn)
/tmp/ipykernel_12980/30002709.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  car_driving_df = pd.read_sql(car_driving, conn)
/tmp/ipykernel_12980/30002709.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlit

In [6]:
print(f'car_siren 데이터 개수: {car_siren_df.shape[0]}')
print(f'car_horn 데이터 개수: {car_horn_df.shape[0]}')
print(f'car_driving 데이터 개수: {car_driving_df.shape[0]}')
print(f'motorcycle_horn 데이터 개수: {motorcycle_horn_df.shape[0]}')
print(f'motorcycle_driving 데이터 개수: {motorcycle_driving_df.shape[0]}')

car_siren 데이터 개수: 1990
car_horn 데이터 개수: 3189
car_driving 데이터 개수: 1682
motorcycle_horn 데이터 개수: 4560
motorcycle_driving 데이터 개수: 4735


In [7]:
#데이터프레임 병합 
df_combined = pd.concat([car_horn_df, car_siren_df,car_driving_df,motorcycle_horn_df,motorcycle_driving_df], ignore_index=True)

print(f'df_combined 행 개수: {len(df_combined)}') 
df_combined.head()

df_combined 행 개수: 16156


,fileName,labelName,area_start,area_end,category_01,category_02,category_03,decibel,soundQuality,subCategory,...,mfcc_41,mfcc_42,mfcc_43,mfcc_44,mfcc_45,mfcc_46,mfcc_47,mfcc_48,mfcc_49,mfcc_50
0,1.car_horn_87900_1.wav,1.car_horn_87900_1.wav,2.0,58.719,교통소음,자동차,차량경적,116,정상,소형차경적,...,-1.194056,-0.757743,-2.858810,-5.101422,-0.198150,7.001447,6.203896,-2.862313,-10.336964,-8.593492
1,1.car_horn_87964_1.wav,1.car_horn_87964_1.wav,2.0,59.747,교통소음,자동차,차량경적,102,정상,소형차경적,...,-7.923782,8.347244,-2.440833,-17.408949,-5.891588,22.778133,21.374588,-19.310514,-26.457247,4.794274
2,1.car_horn_87987_1.wav,1.car_horn_87987_1.wav,2.0,59.217,교통소음,자동차,차량경적,112,정상,소형차경적,...,-5.953082,1.538178,-0.032047,-10.759938,-10.296665,14.938017,19.099138,-5.256958,-27.792421,-7.377015
3,1.car_horn_88000_1.wav,1.car_horn_88000_1.wav,2.0,57.389,교통소음,자동차,차량경적,112,정상,소형차경적,...,-5.018665,2.996135,-0.142866,-8.995171,-4.227169,8.116351,11.163322,-4.992955,-16.359087,-5.441961
4,1.car_horn_88001_1.wav,1.car_horn_88001_1.wav,2.0,58.207,교통소음,자동차,차량경적,122,정상,소형차경적,...,-5.330876,2.107300,1.768516,-10.489394,-4.815802,9.592706,12.866017,-3.469832,-20.286922,-6.563427


In [18]:
# 데이터프레임을 CSV 파일로 저장
df_combined.to_csv('train_data.csv', index=False, encoding='utf-8-sig')

print("데이터프레임이 csv 파일로 저장되었습니다.")

데이터프레임이 csv 파일로 저장되었습니다.


## 2. 기타소음 데이터 프레임 생성

In [8]:
import os
import json
import pandas as pd

def load_and_flatten_json(root_dir):
    data = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.json'):
                filepath = os.path.join(subdir, file)
                with open(filepath, 'r', encoding='utf-8') as f:
                    try:
                        json_data = json.load(f)
                        info = json_data.get('info', {})
                        license_info = json_data.get('license', {})
                        audio = json_data.get('audio', {})
                        environment = json_data.get('environment', {})
                        annotations = json_data.get('annotations', [])

                        area_start = None
                        area_end = None
                        category_01 = None
                        category_02 = None
                        category_03 = None
                        labelName = None
                        soundQuality = None
                        decibel = None
                        subCategory = None

                        if annotations and isinstance(annotations, list) and len(annotations) > 0:
                            annotation = annotations[0]
                            if isinstance(annotation, dict):
                                area = annotation.get('area', {})
                                area_start = area.get('start')
                                area_end = area.get('end')
                                categories = annotation.get('categories', {})
                                category_01 = categories.get('category_01')
                                category_02 = categories.get('category_02')
                                category_03 = categories.get('category_03')
                                labelName = annotation.get('labelName')
                                soundQuality = annotation.get('soundQuality')
                                decibel = annotation.get('decibel')
                                subCategory = annotation.get('subCategory')

                        row = {
                            'area_start': area_start,
                            'area_end': area_end,
                            'category_01': category_01,
                            'category_02': category_02,
                            'category_03': category_03,
                            'decibel': decibel,
                            'labelName': labelName,
                            'soundQuality': soundQuality,
                            'subCategory': subCategory,
                            'bitRate': audio.get('bitRate'),
                            'duration': audio.get('duration'),
                            'fileFormat': audio.get('fileFormat'),
                            'fileName': audio.get('fileName'),
                            'fileSize': audio.get('fileSize'),
                            'recodingType': audio.get('recodingType'),
                            'sampleRate': audio.get('sampleRate'),
                            'acqDevice': environment.get('acqDevice'),
                            'acqMethod': environment.get('acqMethod'),
                            'acqType': environment.get('acqType'),
                            'areaUse': environment.get('areaUse'),
                            'dayNight': environment.get('dayNight'),
                            'direction': environment.get('direction'),
                            'distance': environment.get('distance'),
                            'district': environment.get('district'),
                            'latitude': environment.get('gps', {}).get('latitude'),
                            'longitude': environment.get('gps', {}).get('longitude'),
                            'micClass': environment.get('micClass'),
                            'obstacle': environment.get('obstacle'),
                            'place': environment.get('place'),
                            'recordingTime': environment.get('recordingTime'),
                            'urban': environment.get('urban'),
                            'weather': environment.get('weather'),
                            'contributor': info.get('contributor'),
                            'dateCreated': info.get('dateCreated'),
                            'description': info.get('description'),
                            'uri': info.get('uri'),
                            'version': info.get('version'),
                            'year': info.get('year'),
                            'license_name': license_info.get('name'),
                            'license_url': license_info.get('url'),
                        }
                        data.append(row)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON file: {filepath} - {e}")
    return pd.DataFrame(data)

# JSON 파일 로드 및 평탄화
root_dir_2 = '/home/ubuntu/data/etc_noise_data_test_04'
etc_json_df = load_and_flatten_json(root_dir_2)

print("\n기타 소음 JSON 데이터프레임:")
print(etc_json_df.head())
print(etc_json_df.info())


기타 소음 JSON 데이터프레임:
   area_start  area_end category_01 category_02 category_03  decibel  \
0        4.47    62.220        생활소음          가전         청소기       51   
1        0.35    58.630        생활소음          가전         청소기       79   
2        0.17    30.360        생활소음          가전         청소기      117   
3        0.05    59.000        생활소음          가전         청소기       87   
4        2.00    22.558        생활소음          가전         청소기       49   

          labelName soundQuality subCategory   bitRate  ...  urban weather  \
0  6.가전_76225_1.wav           정상         청소기   705kbps  ...  서울특별시      맑음   
1  6.가전_47589_1.wav           정상         청소기   705kbps  ...  서울특별시      맑음   
2  6.가전_45929_1.wav           정상         청소기  1411kbps  ...  서울특별시      맑음   
3  6.가전_77080_1.wav           정상         청소기  1411kbps  ...  인천광역시      맑음   
4   6.가전_5390_1.wav           정상         청소기  1411kbps  ...  서울특별시      흐림   

  contributor  dateCreated description                      uri version  year 

In [9]:
import librosa
import numpy as np
import os
import pandas as pd
import re
import json

# 한글-영어 매칭 딕셔너리 보완
kor_eng_dict = {
    "자동차": "car",
    "이륜자동차": "motorcycle",
    "경적": "horn",
    "사이렌": "siren",
    "주행음": "driving_sound",
    # 필요한 모든 매핑 추가
}

# 파일명 정규화 함수 수정
def normalize_filename(filename):
    # 확장자 제거
    name, ext = os.path.splitext(filename)

    # 대소문자 변환 및 공백 제거
    name = name.lower().replace(' ', '')

    # 한글-영어 변환 적용
    for kor, eng in kor_eng_dict.items():
        name = name.replace(kor, eng)

    # 정규식으로 고유 ID 추출
    match = re.match(r".*?_(\d+)(?:_\d+)?$", name)
    if match:
        file_id = match.group(1)
    else:
        file_id = None

    return name, ext, file_id

# 매칭 검사 함수 수정
def find_mismatched_files(directory):
    wav_files = {}
    json_files = {}

    # 모든 폴더 순회하며 파일 수집
    for root, _, files in os.walk(directory):
        for file in files:
            name, ext, file_id = normalize_filename(file)
            if file_id is None:
                continue  # ID가 없는 파일은 패스
            file_path = os.path.join(root, file)
            if ext == ".wav":
                wav_files[file_id] = file_path
            elif ext == ".json":
                json_files[file_id] = file_path

    # 매칭되지 않은 파일 찾기
    mismatched_wav = set(wav_files.keys()) - set(json_files.keys())
    mismatched_json = set(json_files.keys()) - set(wav_files.keys())

    return mismatched_wav, mismatched_json, wav_files, json_files

def extract_mfcc(wav_path):
    """
    주어진 WAV 파일 경로에서 MFCC 특징을 추출합니다.
    """
    try:
        audio, sr = librosa.load(wav_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=50)
        mfcc_mean = np.mean(mfcc, axis=1).tolist()
        return mfcc_mean
    except Exception as e:
        print(f"Error processing WAV file: {wav_path} - {e}")
        return None

def load_wav_and_extract_mfcc(root_dir, df_json):
    """
    주어진 디렉토리의 모든 WAV 파일을 로드하고 MFCC 특징을 추출하여 DataFrame에 추가합니다.
    """
    mfcc_data = []
    for index, row in df_json.iterrows():
        wav_filename = row['fileName']
        wav_path = None

        # JSON 파일 이름 정규화
        json_name, _, json_id = normalize_filename(wav_filename)

        # 모든 하위 디렉토리에서 WAV 파일 검색
        for subdir, dirs, files in os.walk(root_dir):
            for file in files:
                wav_name, _, wav_id = normalize_filename(file)
                if json_id == wav_id and file.endswith('_1.wav'):  # ID가 일치하고 _1로 끝나는 파일
                    wav_path = os.path.join(subdir, file)
                    break

        if wav_path:
            mfcc_features = extract_mfcc(wav_path)
            mfcc_data.append(mfcc_features)
        else:
            print(f"WAV file not found for: {wav_filename}, looking for ID: {json_id}")
            mfcc_data.append(None)

    mfcc_columns = [f'mfcc_{i+1}' for i in range(50)]
    df_mfcc = pd.DataFrame(mfcc_data, columns=mfcc_columns)
    return pd.concat([df_json, df_mfcc], axis=1)

# JSON 파일 로드 및 평탄화
root_dir_2 = '/home/ubuntu/data/etc_noise_data_test_04'
etc_json_df = load_and_flatten_json(root_dir_2)

# WAV 파일 로드 및 MFCC 추출
df_final_2 = load_wav_and_extract_mfcc(root_dir_2, etc_json_df)
print(df_final_2.head())

   area_start  area_end category_01 category_02 category_03  decibel  \
0        4.47    62.220        생활소음          가전         청소기       51   
1        0.35    58.630        생활소음          가전         청소기       79   
2        0.17    30.360        생활소음          가전         청소기      117   
3        0.05    59.000        생활소음          가전         청소기       87   
4        2.00    22.558        생활소음          가전         청소기       49   

          labelName soundQuality subCategory   bitRate  ...   mfcc_41  \
0  6.가전_76225_1.wav           정상         청소기   705kbps  ... -1.863527   
1  6.가전_47589_1.wav           정상         청소기   705kbps  ... -3.491574   
2  6.가전_45929_1.wav           정상         청소기  1411kbps  ... -0.153434   
3  6.가전_77080_1.wav           정상         청소기  1411kbps  ... -2.305894   
4   6.가전_5390_1.wav           정상         청소기  1411kbps  ...  4.008397   

    mfcc_42   mfcc_43   mfcc_44   mfcc_45   mfcc_46   mfcc_47   mfcc_48  \
0  8.110558  0.582024 -3.542713 -1.612815  3.155624 -

## 3. 최종 데이터프레임 생성

In [10]:
# df_combined의 열 목록 가져오기
combined_cols = df_combined.columns.tolist()

# df_final_2에서 df_combined에 있는 열만 선택
final_cols = [col for col in combined_cols if col in df_final_2.columns]
df_final_2 = df_final_2[final_cols]

# df_final_2의 열 순서를 df_combined의 열 순서에 맞춰 변경
df_final_2 = df_final_2[combined_cols[:len(final_cols)]]

# 두 데이터 프레임 합치기
df_result = pd.concat([df_combined, df_final_2], axis=0)

print(df_result)

                    fileName               labelName  area_start  area_end  \
0     1.car_horn_87900_1.wav  1.car_horn_87900_1.wav        2.00    58.719   
1     1.car_horn_87964_1.wav  1.car_horn_87964_1.wav        2.00    59.747   
2     1.car_horn_87987_1.wav  1.car_horn_87987_1.wav        2.00    59.217   
3     1.car_horn_88000_1.wav  1.car_horn_88000_1.wav        2.00    57.389   
4     1.car_horn_88001_1.wav  1.car_horn_88001_1.wav        2.00    58.207   
...                      ...                     ...         ...       ...   
1860          7.동물_21233.wav        7.동물_21233_1.wav        2.00    22.464   
1861          7.동물_49714.wav        7.동물_49714_1.wav        2.00    12.311   
1862          7.동물_50077.wav        7.동물_50077_1.wav        2.00    26.916   
1863          7.동물_49929.wav        7.동물_49929_1.wav        2.00    29.847   
1864          7.동물_93791.wav        7.동물_93791_1.wav        2.55     5.780   

     category_01 category_02 category_03  decibel soundQuality 

In [11]:
# 각 열의 null 값 개수 확인
print(df_result.isnull().sum())

# 데이터프레임 전체에 null 값 있는지 확인
print(df_result.isnull().values.any())

fileName       0
labelName      0
area_start     0
area_end       0
category_01    0
              ..
mfcc_46        0
mfcc_47        0
mfcc_48        0
mfcc_49        0
mfcc_50        0
Length: 82, dtype: int64
False


In [12]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18021 entries, 0 to 1864
Data columns (total 82 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   fileName       18021 non-null  object 
 1   labelName      18021 non-null  object 
 2   area_start     18021 non-null  float64
 3   area_end       18021 non-null  float64
 4   category_01    18021 non-null  object 
 5   category_02    18021 non-null  object 
 6   category_03    18021 non-null  object 
 7   decibel        18021 non-null  int64  
 8   soundQuality   18021 non-null  object 
 9   subCategory    18021 non-null  object 
 10  bitRate        18021 non-null  object 
 11  duration       18021 non-null  float64
 12  fileFormat     18021 non-null  object 
 13  fileSize       18021 non-null  int64  
 14  recodingType   18021 non-null  object 
 15  sampleRate     18021 non-null  object 
 16  acqDevice      18021 non-null  object 
 17  acqMethod      18021 non-null  object 
 18  acqType     

In [13]:
print(df_result['category_03'].value_counts())

category_03
이륜차주행음    4735
이륜차경적     4560
차량경적      3189
차량사이렌     1990
차량주행음     1682
헬리콥터       435
기차         281
고양이        274
지하철        245
개          217
비행기        191
세탁기        120
청소기        102
Name: count, dtype: int64


## 4. 기타소음 분류 컬럼 추가

In [14]:
# df_result에서 실제 라벨을 기준으로 정상 데이터와 기타 소음 라벨링
valid_labels = ['차량경적', '차량주행음', '이륜차경적', '이륜차주행음', '차량사이렌']  # 정상 라벨

# 'category_03' 컬럼이 정상 라벨이면 그대로 두고, 그렇지 않으면 '기타소음'으로 설정
df_result['ood_label'] = df_result['category_03'].apply(lambda x: x if x in valid_labels else '기타소음')

In [15]:
print(df_result['category_03'].value_counts())

category_03
이륜차주행음    4735
이륜차경적     4560
차량경적      3189
차량사이렌     1990
차량주행음     1682
헬리콥터       435
기차         281
고양이        274
지하철        245
개          217
비행기        191
세탁기        120
청소기        102
Name: count, dtype: int64


In [16]:
print(df_result['ood_label'].value_counts())

ood_label
이륜차주행음    4735
이륜차경적     4560
차량경적      3189
차량사이렌     1990
기타소음      1865
차량주행음     1682
Name: count, dtype: int64


## 5. csv 파일로 저장

In [17]:
# 데이터프레임을 CSV 파일로 저장
df_result.to_csv('combined_result.csv', index=False, encoding='utf-8-sig')

print("데이터프레임이 csv 파일로 저장되었습니다.")

데이터프레임이 csv 파일로 저장되었습니다.
